In [ ]:
!git clone https://github.com/CornerSiow/zero-shot-image-captioning.git
!pip install pycocoevalcap

In [ ]:
import re
import io
import json
import sys
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

In [ ]:
def generateReferenceFile(filename):
  info = {"year" : 2022, "version" : '1',
            "description" : 'Evaluate Captions',
            "contributor" : 'Corner Siow',
            "url" : 'https://github.com/CornerSiow/zero-shot-image-captioning',
            "date_created" : ''}
  licenses = [{"id" : 1,"name" : "","url" : "",}]
  res = {"info" : info, "type" : 'captions',
         "images" :  [], "annotations" : [],
         "licenses" : licenses,}
  count = 0
  images = []
  annotations = []

  with open(filename,'r') as opfd:
    for line in opfd:
      count +=1
      data = line.strip().split('\t')
      assert len(data)==2
      caption = data[1]
      id = count
      
      images.append({
          "id" : id, "width" : 0, "height" : 0, 
          "file_name" : data[0], "license" : '', 
          "url" : data[0], "date_captured" : ''})
      annotations.append({
          "id" : len(annotations)+1,
          "image_id" : id,
          "caption" : caption,
          })
  res["images"] = images
  res["annotations"] = annotations
  outputFileName = filename.split('/')[-1].split('.')[0] + ".json"
  with io.open(outputFileName, 'w', encoding='utf-8') as fd:
      fd.write(json.dumps(res, ensure_ascii=True,sort_keys=True,indent=2,separators=(',', ': ')))
  imagesIDList = {}
  for v in images:
    imagesIDList[v['file_name']] = v['id']
  return imagesIDList, outputFileName

In [ ]:
def generatePredictionFile(filename, imagesIDList):
  count = 0
  res = []
  with open(filename,'r') as opfd:
    for line in opfd:
      count +=1
      data = line.strip().split('\t')
      assert len(data) == 2
      caption = data[1]
      id = imagesIDList[data[0]]
      res.append({"image_id":id,"caption":caption})
  outputFileName = filename.split('/')[-1].split('.')[0] + ".json"
  with io.open(outputFileName, 'w', encoding='utf-8') as fd:
      fd.write(json.dumps(res, ensure_ascii=True,sort_keys=True,indent=2,separators=(',', ': ')))
  return outputFileName

In [ ]:
imagesIDList, referenceFile = generateReferenceFile("/content/zero-shot-image-captioning/data/image_caption.txt")
predictionFile = generatePredictionFile("/content/zero-shot-image-captioning/data/captioning_result_with_RVS.txt", imagesIDList)    
# create coco object and cocoRes object
coco = COCO(referenceFile)
cocoRes = coco.loadRes(predictionFile)
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)
# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
cocoEval.evaluate()
# print output evaluation scores
print("---------------------------------------------------------")
for metric, score in cocoEval.eval.items():
    print ('%s: %.3f'%(metric, score))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 84, 'reflen': 84, 'guess': [84, 74, 64, 54], 'correct': [84, 74, 64, 54]}
ratio: 0.9999999999880952
Bleu_1: 1.000
Bleu_2: 1.000
Bleu_3: 1.000
Bleu_4: 1.000
computing METEOR score...
METEOR: 1.000
computing Rouge score...
ROUGE_L: 1.000
computing CIDEr score...
CIDEr: 10.000
computing SPICE score...
SPICE: 1.000
---------------------------------------------------------
Bleu_1: 1.000
Bleu_2: 1.000
Bleu_3: 1.000
Bleu_4: 1.000
METEOR: 1.000
ROUGE_L: 1.000
CIDEr: 10.000
SPICE: 1.000


In [ ]:
imagesIDList, referenceFile = generateReferenceFile("/content/zero-shot-image-captioning/data/image_caption.txt")
predictionFile = generatePredictionFile("/content/zero-shot-image-captioning/data/captioning_result_without_RVS.txt", imagesIDList)    
# create coco object and cocoRes object
coco = COCO(referenceFile)
cocoRes = coco.loadRes(predictionFile)
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)
# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
cocoEval.evaluate()
# print output evaluation scores
print("---------------------------------------------------------")
for metric, score in cocoEval.eval.items():
    print ('%s: %.3f'%(metric, score))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 88, 'reflen': 84, 'guess': [88, 78, 68, 58], 'correct': [78, 64, 52, 44]}
ratio: 1.0476190476065759
Bleu_1: 0.886
Bleu_2: 0.853
Bleu_3: 0.822
Bleu_4: 0.806
computing METEOR score...
METEOR: 0.585
computing Rouge score...
ROUGE_L: 0.891
computing CIDEr score...
CIDEr: 8.099
computing SPICE score...
SPICE: 0.889
---------------------------------------------------------
Bleu_1: 0.886
Bleu_2: 0.853
Bleu_3: 0.822
Bleu_4: 0.806
METEOR: 0.585
ROUGE_L: 0.891
CIDEr: 8.099
SPICE: 0.889
